<a href="https://colab.research.google.com/github/stemike/mimic-lstm/blob/master/dataset_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import cv2
import json
import os
import pickle
import torch

import matplotlib.pyplot as plt
import numpy as np

from google.colab.patches import cv2_imshow
from glob import glob

In [4]:
# constants
drive_path = '/content/drive/My Drive'
user_path = '/JKU/CV'
project_path = '/CV_Project/Dataset/data_WiSAR/data/'
impath = drive_path + user_path + project_path

data_folders = ['train/', 'validation/', 'test/']
mask_path = impath + 'mask.png'
homography_name = 'homographies.json'
mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE) # Grayscale mask

In [4]:
# Get an overview of the amount of data
for data_folder in data_folders:
  samples = glob(f'{impath + data_folder}/*/')
  print(f'{len(samples)} samples in {data_folder}')
  n_images = [len(glob(f'{sample}/*.png')) for sample in samples]
  print(f'Average of {np.mean(n_images)} images with std of {np.std(n_images)} per sample in {data_folder}')

26 samples in train/
Average of 70.0 images with std of 0.0 per sample in train/
11 samples in validation/
Average of 70.0 images with std of 0.0 per sample in validation/
13 samples in test/
Average of 70.0 images with std of 0.0 per sample in test/


In [5]:
def merge_cameras(image_paths, homographies, mask=None):
  """
  Merges the 10 cameras of a sample into one image
  """
  images = []
  for image_path in image_paths:
    # Extracts the file name from full path. 
    image_key = image_path.split('/')[-1].split('.')[0]

    image = cv2.imread(image_path)
    if mask is not None:
      image = cv2.bitwise_and(image, image, mask = mask)
    homography = np.array(homographies[image_key])
    warped_image = cv2.warpPerspective(image, homography, image.shape[:2])
    images.append(warped_image)
  
  return np.mean(images, axis=0)

def draw_boxes_on_val_image(image, sample_name, bb_path, show_small=True):
  """
  Draws the ground truth boxes on an image
  """
  with open(bb_path, 'r') as f:
    gtlabels = json.load(f)

  for bb in gtlabels[sample_name]:
    x,y,w,h = bb
    image = cv2.rectangle(image,(x, y),(x+w, y+h),(0,0,255),5)
  
  if show_small:
    image = cv2.resize(image, (500, 500))

  cv2_imshow(image)

def prepare_dataset(data_folder, homography_name, show_images=False, n_max_frame=7):
  """
  Returns an array of shape N_Sample x N_Frames X Image_Shape
  """
  print(data_folder)
  samples = []
  for sample in sorted(glob(f'{data_folder}/*/')):
    homography_path = f'{sample}{homography_name}'
    with open(homography_path, 'r') as f:
      homographies = json.load(f)
    sample_frames = []

    print(f'\r{sample}', end='')

    for frame in range(n_max_frame):
      images = []
      image_paths = glob(f'{sample}/{frame}-*.png')

      image = merge_cameras(image_paths, homographies, mask)
      
      if frame == 0 and show_images:
        if 'validation' in data_folder:
          bb_path = f'{data_folder}/labels.json'
          sample_name = sample.split('/')[-2]
          draw_boxes_on_val_image(image, sample_name, bb_path, show_small=True)
        else:
          image_small = cv2.resize(image, (500, 500))
          cv2_imshow(image_small)
      
      sample_frames.append(image)
    samples.append(np.array(sample_frames))
  print()
  return np.array(samples)

In [6]:
for data_folder in data_folders:
  set_name = data_folder.split('/')[-2]
  np_dataset = prepare_dataset(impath + data_folder, homography_name)
  print(np_dataset.shape, end=' to ')
  np_dataset = np_dataset.reshape(-1, np_dataset.shape[-3], np_dataset.shape[-2], np_dataset.shape[-1])
  print(np_dataset.shape)
  labels = torch.ones(np_dataset.shape[0])
  tensor_dataset = torch.utils.data.TensorDataset(torch.Tensor(np_dataset), labels)
  
  with open(f'{impath}{set_name}.pickle', 'wb') as f:
    print(f'Saved to dataset to {impath}{set_name}.pickle')
    pickle.dump(tensor_dataset, f)

/content/drive/My Drive/JKU/CV/CV_Project/Dataset/data_WiSAR/data/train/
/content/drive/My Drive/JKU/CV/CV_Project/Dataset/data_WiSAR/data/train/train-2-9/
(26, 7, 1024, 1024, 3) to (182, 1024, 1024, 3)
Saved to dataset to /content/drive/My Drive/JKU/CV/CV_Project/Dataset/data_WiSAR/data/train.pickle
/content/drive/My Drive/JKU/CV/CV_Project/Dataset/data_WiSAR/data/validation/
/content/drive/My Drive/JKU/CV/CV_Project/Dataset/data_WiSAR/data/validation/valid-2-3/
(11, 7, 1024, 1024, 3) to (77, 1024, 1024, 3)
Saved to dataset to /content/drive/My Drive/JKU/CV/CV_Project/Dataset/data_WiSAR/data/validation.pickle
/content/drive/My Drive/JKU/CV/CV_Project/Dataset/data_WiSAR/data/test/
/content/drive/My Drive/JKU/CV/CV_Project/Dataset/data_WiSAR/data/test/test-2-8/
(13, 7, 1024, 1024, 3) to (91, 1024, 1024, 3)
Saved to dataset to /content/drive/My Drive/JKU/CV/CV_Project/Dataset/data_WiSAR/data/test.pickle


In [5]:
show_pictures = False
if show_pictures:
  prepare_dataset(impath + data_folders[1], homography_name, show_images=True)
  print()
else:
  load_path = f'{impath}train.pickle'
  with open(load_path, 'rb') as f:
    print(f'Loading file from: {load_path}')
    ds = pickle.load(f, encoding= 'unicode_escape') 
    print(ds)
  dl = torch.utils.data.DataLoader(ds, batch_size=16)
  print(dl)

Loading file from: /content/drive/My Drive/JKU/CV/CV_Project/Dataset/data_WiSAR/data/train.pickle
